![image](https://images.theconversation.com/files/351698/original/file-20200807-18-13k2a9d.jpg?ixlib=rb-1.1.0&rect=0%2C9%2C2158%2C1053&q=20&auto=format&w=320&fit=clip&dpr=2&usm=12&cs=strip)
# **<span style = "color:#008514"> Introduction</span>**
### The usage of social media has been increased by few times in the recent years. With this increasing number of users and content posted by them the interest of automatically identifying the users  has also increased. behaviour pattern such as political inclination, personality are the things we are talking about. Other things such as Gender, Ethnicity, age are also interesting. In this task we will focus on detecting the Gender of twitter users by their tweet written in Portuguese by extracting gender expression linguistic cues utilizing 25 attributes. 

# **<span style = "color:#008514 ">Problem Statement</span>**
### Predict user gender based on twitter gender Information.

# **<span style = "color:#008514 ">Data Source</span>**
### This data is extracted from Kaggle. The link to the page is [Here](https://www.kaggle.com/datasets/crowdflower/twitter-user-gender-classification)
### The information about the features of the dataset in given there.

# **<span style = "color:#008514 ">Import all the necesary libraries and Classifiers</span>**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
import nltk
import re
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
data = pd.read_csv('../input/twitter-user-gender-classification/gender-classifier-DFE-791531.csv',encoding='latin1')

# **<span style = "color:#008514 ">Data Exploration</span>**

In [3]:
data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN


In [4]:
data.shape

(20050, 26)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20050 entries, 0 to 20049
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _unit_id               20050 non-null  int64  
 1   _golden                20050 non-null  bool   
 2   _unit_state            20050 non-null  object 
 3   _trusted_judgments     20050 non-null  int64  
 4   _last_judgment_at      20000 non-null  object 
 5   gender                 19953 non-null  object 
 6   gender:confidence      20024 non-null  float64
 7   profile_yn             20050 non-null  object 
 8   profile_yn:confidence  20050 non-null  float64
 9   created                20050 non-null  object 
 10  description            16306 non-null  object 
 11  fav_number             20050 non-null  int64  
 12  gender_gold            50 non-null     object 
 13  link_color             20050 non-null  object 
 14  name                   20050 non-null  object 
 15  pr

# **<span style = "color:#008514 ">Data Cleaning</span>**

### we will be droping the redundant columns from our data

In [6]:
data.drop(['_unit_id','_last_judgment_at','created','fav_number','profileimage','retweet_count','tweet_coord',
         '_trusted_judgments', 'tweet_count', 'tweet_created', 'tweet_id', 'tweet_location', 'user_timezone', 
         '_golden','_unit_state', 'gender_gold', 'link_color', 'name', 'profile_yn_gold', 'sidebar_color', 
         'profile_yn', 'profile_yn:confidence','gender:confidence'], axis=1, inplace=True)

In [7]:
data.head()

,gender,description,text
0,male,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...
1,male,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...
2,male,louis whining and squealing and all,i absolutely adore when louis starts the songs...
3,male,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...
4,female,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...


### we will check for null values

In [8]:
data.isna().sum()

gender           97
description    3744
text              0
dtype: int64

### Will drop the rows with null values

In [9]:
data.dropna(axis = 0, inplace = True)

### Let's count the gender column

In [10]:
data['gender'].value_counts()

female     5725
male       5469
brand      4328
unknown     702
Name: gender, dtype: int64

### we will only keep the male and female variables and drop the others.

In [11]:
data = data[(data.gender == 'female') | (data.gender == 'male')]

### And check the counts again

In [12]:
data['gender'].value_counts()

female    5725
male      5469
Name: gender, dtype: int64

# **<span style = "color:#008514 ">Data Preprocessing</span>**

### we will encode male and female category as 1 and 0

In [13]:
for gen in data['gender']:
    if gen == 'male':
        data['gender'].replace({'male':1}, inplace = True)
    elif gen == 'female':
        data['gender'].replace({'female':0}, inplace = True)

data['gender'].value_counts()

0    5725
1    5469
Name: gender, dtype: int64

## **<span style = "color:#2de34f ">Cleaning Text</span>**

### we will be cleaning the description column

In [14]:
def clean(review):

    descrip = re.sub('[^a-zA-Z]', ' ', review)

    review = review.lower()
    return review

data['descrip_cleaned'] = pd.DataFrame(data['description'].apply(lambda x: clean(x)))

data.head()

,gender,description,text,descrip_cleaned
0,1,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,i sing my own rhythm.
1,1,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...,i'm the author of novels filled with family dr...
2,1,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining and squealing and all
3,1,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,"mobile guy. 49ers, shazam, google, kleiner pe..."
4,0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson the best frontman/kaiser chiefs t...


In [15]:
data['descrip_cleaned'].replace('[@+]', "", regex=True,inplace=True)
data['descrip_cleaned'].replace('[()]', "", regex=True,inplace=True)
data['descrip_cleaned'].replace('[#+]', "", regex=True,inplace=True)
url_regex = '''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
data['descrip_cleaned'] = data['descrip_cleaned'].replace(url_regex, "", regex=True)


In [16]:
data.head()

,gender,description,text,descrip_cleaned
0,1,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,i sing my own rhythm.
1,1,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...,i'm the author of novels filled with family dr...
2,1,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining and squealing and all
3,1,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,"mobile guy. 49ers, shazam, google, kleiner pe..."
4,0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson the best frontman/kaiser chiefs t...


## **<span style = "color:#2de34f ">Tokenize the cleaned descriptions</span>**

### Tokenization is the process of breaking text into smaller pieces which we know as tokens. More about Tokenization in this [link](https://www.tokenex.com/resource-center/what-is-tokenization)

In [17]:
data['descrip_cleaned'] = [nltk.word_tokenize(tweet) for tweet in data['descrip_cleaned']]
descrip_new = []
for each_row in data['descrip_cleaned']:
    descrip_new.append([i for i in each_row if i.isalpha()])
descrip_new_alpha = []


## **<span style = "color:#2de34f ">Stopwords Removal</span>**
### stop words don't add any meaning to the sentences. we will remove them

In [18]:
stop_words = set(stopwords.words('english'))
# now remove them from the list
for each_row in descrip_new:
    descrip_new_alpha.append([i for i in each_row if i not in stop_words])

## **<span style = "color:#2de34f ">Lemmatization</span>**

### Lemmatization is the process of grouping together the different inflected forms of a word so they can be analyzed as a single item. Lemmatization is similar to stemming but it brings context to the words. So it links words with similar meanings to one word.

In [19]:
description_new_lemma = []
lemma = nltk.WordNetLemmatizer()
for each_row in descrip_new_alpha:
    description_new_lemma.append([lemma.lemmatize(word) for word in each_row])
#put back the new sentences
data['descrip_cleaned'] = description_new_lemma
data['descrip_cleaned'] = [" ".join(desc) for desc in data['descrip_cleaned'].values]

## **<span style = "color:#2de34f ">Text cleaning of the 'text' column</span>**
### whatever we've done for the 'description' column, will do that for 'text' column too

In [20]:
data['text_cleaned'] = pd.DataFrame(data['text'].apply(lambda y: clean(y)))
data.head()

,gender,description,text,descrip_cleaned,text_cleaned
0,1,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,sing rhythm,robbie e responds to critics after win against...
1,1,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...,author novel filled family drama romance,ûïit felt like they were my friends and i was...
2,1,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining squealing,i absolutely adore when louis starts the songs...
3,1,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,mobile guy shazam google kleiner perkins yahoo...,hi @jordanspieth - looking at the url - do you...
4,0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson best chief best band xxxx thank k...,watching neighbours on sky+ catching up with t...


In [21]:
data['text_cleaned'].replace('[@+]', "", regex=True,inplace=True)
data['text_cleaned'].replace('[()]', "", regex=True,inplace=True)
data['text_cleaned'].replace('[#+]', "", regex=True,inplace=True)
url_regex = '''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
data['text_cleaned'] = data['text_cleaned'].replace(url_regex, "", regex=True)

In [22]:
data['text_cleaned'] = [nltk.word_tokenize(tweet) for tweet in data['text_cleaned']]
text_new = []
for each_row in data['text_cleaned']:
    text_new.append([i for i in each_row if i.isalpha()])
text_new_alpha = []

In [23]:
for each_row in text_new:
    text_new_alpha.append([i for i in each_row if i not in stop_words])

In [24]:
text_new_lemma = []
for each_row in text_new_alpha:
    text_new_lemma.append([lemma.lemmatize(word) for word in each_row])
#put back the new sentences
data['text_cleaned'] = text_new_lemma
data['text_cleaned'] = [" ".join(desc) for desc in data['text_cleaned'].values]

In [25]:
data.head()

,gender,description,text,descrip_cleaned,text_cleaned
0,1,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,sing rhythm,robbie e responds critic win eddie edward worl...
1,1,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...,author novel filled family drama romance,felt like friend living story http retired http
2,1,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining squealing,absolutely adore louis start song hit hard fee...
3,1,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,mobile guy shazam google kleiner perkins yahoo...,hi jordanspieth looking url use ifttt typicall...
4,0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson best chief best band xxxx thank k...,watching neighbour sky catching neighbs xxx xxx


## **<span style = "color:#2de34f ">droping Redundant Columns description and text</span>**

In [26]:
data.drop(['description', 'text'], axis = 1, inplace = True)
data.head()

,gender,descrip_cleaned,text_cleaned
0,1,sing rhythm,robbie e responds critic win eddie edward worl...
1,1,author novel filled family drama romance,felt like friend living story http retired http
2,1,louis whining squealing,absolutely adore louis start song hit hard fee...
3,1,mobile guy shazam google kleiner perkins yahoo...,hi jordanspieth looking url use ifttt typicall...
4,0,ricky wilson best chief best band xxxx thank k...,watching neighbour sky catching neighbs xxx xxx


# **<span style = "color:#008514 ">vectorization of texts</span>**

### Text Vectorization is the process of converting text into numerical representation. 
### countvectorizer is one of the most straightforword vectorization method. we will use that in this project

In [27]:
cv = CountVectorizer(max_features = 1000)
x = cv.fit_transform(data['descrip_cleaned']).toarray()
x1 = cv.fit_transform(data['text_cleaned']).toarray()

### let's save these two set of vectors in two different pandas dataframe

In [28]:
A = pd.DataFrame(x)
B = pd.DataFrame(x1)


### and then concatenate these two dataframes

In [29]:
X = pd.concat([B, A], join = 'outer', axis = 1)
X.shape

(11194, 2000)

### the shape of this new dataframe matches shape of our gender column

In [30]:
data['gender'].shape

(11194,)

### Now build our independent variables and Target variable

In [31]:
x = np.array(X)
y = np.array(data['gender'])

## **<span style = "color:#2de34f ">Split the data into train test set</span>**

In [32]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 0)

# **<span style = "color:#008514 ">Classification</span>**

### we will use thrre classifier for this task. Naive Bayes, Lightgbm and XGBoost

## **<span style = "color:#2de34f ">Naive Bayes</span>**

In [33]:
gnb_model = GaussianNB()
gnb_model.fit(X_train , y_train)
y_pred = gnb_model.predict(X_test)

## **<span style = "color:#67bf78 ">Evaluation</span>**

In [34]:
accuracy = accuracy_score(y_test, y_pred)
print("accuracy: %.2f%%" % (accuracy*100.0))

accuracy: 63.11%


In [35]:
confusion_matrix(y_test, y_pred)

array([[789, 343],
       [483, 624]])

In [36]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.70      0.66      1132
           1       0.65      0.56      0.60      1107

    accuracy                           0.63      2239
   macro avg       0.63      0.63      0.63      2239
weighted avg       0.63      0.63      0.63      2239



## **<span style = "color:#67bf78 ">Hyperparameter optimization</span>**

### we are using grid search for hyperparameter optimizing the hyperparameters of the models

In [37]:
param_grid_nb = {'var_smoothing': np.logspace(0, -9, num = 100)}

In [38]:
nb_model_grid = GridSearchCV(estimator = gnb_model, param_grid = param_grid_nb, verbose = 1, cv = 3, n_jobs = -1)

In [39]:
nb_model_grid.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


GridSearchCV(cv=3, estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': array([1.00000000e+00, 8.11130831e-01, 6.57933225e-01, 5.33669923e-01,
       4.32876128e-01, 3.51119173e-01, 2.84803587e-01, 2.31012970e-01,
       1.87381742e-01, 1.51991108e-01, 1.23284674e-01, 1.00000000e-01,
       8.11130831e-02, 6.57933225e-02, 5.33669923e-02, 4.32876128e-02,
       3.51119173e-02, 2.8480358...
       1.23284674e-07, 1.00000000e-07, 8.11130831e-08, 6.57933225e-08,
       5.33669923e-08, 4.32876128e-08, 3.51119173e-08, 2.84803587e-08,
       2.31012970e-08, 1.87381742e-08, 1.51991108e-08, 1.23284674e-08,
       1.00000000e-08, 8.11130831e-09, 6.57933225e-09, 5.33669923e-09,
       4.32876128e-09, 3.51119173e-09, 2.84803587e-09, 2.31012970e-09,
       1.87381742e-09, 1.51991108e-09, 1.23284674e-09, 1.00000000e-09])},
             verbose=1)

In [40]:
nb_model_grid.best_params_

{'var_smoothing': 0.006579332246575682}

In [41]:
y_pred_hyper = nb_model_grid.predict(X_test)

In [42]:
print(confusion_matrix(y_test, y_pred_hyper))

[[885 247]
 [555 552]]


In [43]:
accuracy = accuracy_score(y_test, y_pred_hyper)
print("accuracy after tuning: %.2f%%" % (accuracy*100.0))

accuracy after tuning: 64.18%


In [44]:
print(classification_report(y_test, y_pred_hyper))

              precision    recall  f1-score   support

           0       0.61      0.78      0.69      1132
           1       0.69      0.50      0.58      1107

    accuracy                           0.64      2239
   macro avg       0.65      0.64      0.63      2239
weighted avg       0.65      0.64      0.63      2239



## **<span style = "color:#2de34f ">Lightgbm Classifier</span>**

In [45]:
lgb_model = LGBMClassifier(max_depth=3)
lgb_model.fit(X_train, y_train)

LGBMClassifier(max_depth=3)

## **<span style = "color:#67bf78 ">Evaluation</span>**

In [46]:
y_pred1 = lgb_model.predict(X_test)

In [47]:
accuracy = accuracy_score(y_test, y_pred1)
print("accuracy: %.2f%%" % (accuracy*100.0))

accuracy: 60.92%


In [48]:
confusion_matrix(y_test, y_pred1)

array([[995, 137],
       [738, 369]])

In [49]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.70      0.66      1132
           1       0.65      0.56      0.60      1107

    accuracy                           0.63      2239
   macro avg       0.63      0.63      0.63      2239
weighted avg       0.63      0.63      0.63      2239



## **<span style = "color:#67bf78 ">Hyperparameter optimization</span>**

In [50]:
param_grid_lgb = {
              "max_depth": [2, 3, 5, 10],
              "min_child_weight": [0.001, 0.002],
              "learning_rate": [0.05, 0.1]
              }

In [51]:
lgb_grid = GridSearchCV(estimator = lgb_model, param_grid = param_grid_lgb, verbose = 1, cv = 3, n_jobs = -1)

In [52]:
lgb_grid.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


GridSearchCV(cv=3, estimator=LGBMClassifier(max_depth=3), n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1],
                         'max_depth': [2, 3, 5, 10],
                         'min_child_weight': [0.001, 0.002]},
             verbose=1)

In [53]:
lgb_grid.best_params_

{'learning_rate': 0.1, 'max_depth': 10, 'min_child_weight': 0.001}

In [54]:
y_pred1_hyper = lgb_grid.predict(X_test)

In [55]:
accuracy = accuracy_score(y_test, y_pred1_hyper)
print("accuracy after tuning: %.2f%%" % (accuracy*100.0))

accuracy after tuning: 62.44%


In [56]:
print(confusion_matrix(y_test, y_pred1_hyper))

[[901 231]
 [610 497]]


In [57]:
print(classification_report(y_test, y_pred1_hyper))

              precision    recall  f1-score   support

           0       0.60      0.80      0.68      1132
           1       0.68      0.45      0.54      1107

    accuracy                           0.62      2239
   macro avg       0.64      0.62      0.61      2239
weighted avg       0.64      0.62      0.61      2239



## **<span style = "color:#2de34f ">XGBoost Classifier</span>**

In [58]:
xgb_model = XGBClassifier(max_depth = 5, min_child_weight = 1)
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

## **<span style = "color:#67bf78 ">Evaluation</span>**

In [59]:
y_pred2 = xgb_model.predict(X_test)

In [60]:
accuracy = accuracy_score(y_test, y_pred2)
print('Accuracy: %.2f%%' % (accuracy * 100.0))

Accuracy: 63.42%


In [61]:
confusion_matrix(y_test, y_pred2)

array([[870, 262],
       [557, 550]])

In [62]:
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.61      0.77      0.68      1132
           1       0.68      0.50      0.57      1107

    accuracy                           0.63      2239
   macro avg       0.64      0.63      0.63      2239
weighted avg       0.64      0.63      0.63      2239



## **<span style = "color:#67bf78 ">Hyperparameter optimization</span>**

In [63]:
xgb_param_grid = {
              "max_depth": [3, 5],
              "min_child_weight": [1, 2],
              }

In [64]:
xgb_grid = GridSearchCV(estimator = xgb_model, param_grid = xgb_param_grid, verbose = 1, cv = 3, n_jobs = -1)

In [65]:
xgb_grid.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     callbacks=None, colsample_bylevel=1,
                                     colsample_bynode=1, colsample_bytree=1,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=0, gpu_id=-1,
                                     grow_policy='depthwise',
                                     importance_type=None,
                                     interaction_constraints='',
                                     learning_rate=0.300000012, max_bin=256,
                                     max_cat_to_onehot=4, max_delta_step=0,
                                     max_depth=5, max_leaves=0,
                                     min_child_weight=1, missing=nan,
                                     monotone_constraints='()',
                   

In [66]:
xgb_grid.best_params_

{'max_depth': 5, 'min_child_weight': 2}

In [67]:
y_pred2_hyper = xgb_grid.predict(X_test)

In [68]:
accuracy = accuracy_score(y_test, y_pred2_hyper)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 63.78%


In [69]:
confusion_matrix(y_test, y_pred2_hyper)

array([[880, 252],
       [559, 548]])

In [70]:
print(classification_report(y_test, y_pred2_hyper))

              precision    recall  f1-score   support

           0       0.61      0.78      0.68      1132
           1       0.69      0.50      0.57      1107

    accuracy                           0.64      2239
   macro avg       0.65      0.64      0.63      2239
weighted avg       0.65      0.64      0.63      2239



# **<span style = "color:#008514 ">Conclusion</span>**

### the result of the XGBClassifier did'nt improve even after hyperparameter tuning, but for naive Bayes Classifier and LightGBM Classifier it improved.
### Naive Bayes proved to be the best performing model for this dataset with an accuracy of 64.18%.

## **<span style = "color: #03ff33">Thanks for visiting</span>**